# <center >谷歌开源Agent框架ADK开发实战</center>
## <center>Part 3. ADK 代理运行时及事件类型详解</center>

&emsp;&emsp;`Google Agent Development Kit`是目前相较于其他`Agent`开发框架如`LangGraph`、`AutoGen`等，其设计理念最前沿的一个`Agent`开发框架，同时与`OpenAI`的`Agents SDK`框架相比，整体架构非常类似，但其模块化的程度更高，原生支持企业应用开发场景的组件也更多，综合来看，`ADK`框架是当前最值得推荐学习和使用的`Agent`开发框架，没有之一。

&emsp;&emsp;正如在前两节`ADK`的快速入门课程中，大家能清楚明确的是当我们想要使用`ADK`框架开发一个`Agent`应用时，一个基本的构建流程如下：

```python
    # 1. 通过 litellm 接入模型
    model = LiteLlm(...)

    # 2. 通过 ADK 的 Agent 组件定义一个 Agent
    agent = Agent(model=model, tools=tools)

    # 3. 构建 session
    session_service = InMemorySessionService()

    # 4. 构建Runner对象
    runner = Runner(agent=agent, session_service=session_service)

    # 5. 运行Agent
    response = runner.run(user_input)
```

&emsp;&emsp;上述伪代码就是在`ADK`框架中，构建一个`Agent`应用的最简、且必备的五步流程。但这几行简单的代码调用其背后是有一套完整的事件驱动机制来支撑。所以大家要做好准备，`ADK`是截止目前相较于之前讲解的`Agent`开发框架中最复杂的一个。因此，接下来的课程中，我们将会从`ADK`的架构设计、事件驱动机制、组件的实现原理、以及如何基于`ADK`框架开发一个企业级的`Agent`应用等维度，展开详细的讲解。

# 一、ADK 架构设计解析

&emsp;&emsp;对新兴一代的`Agent`开发框架，其核心且必须具备的两个能力：<font color=red>其一是在构建`Agent`的时候能够灵活的赋予其工具调用/`MCP Servers`，其二是可以灵活的与其他`Agent`进行协作。</font> `ADK` 框架则是采用模块化设计，依次通过**Agents**模块、**Tools**模块、**Sessions/State**模块，以及回调与事件等组件构成的**Runner**模块来实现上述两个核心能力。

&emsp;&emsp;我们可以先从官方提供的`ADK`架构图来了解一下`ADK`的架构设计。从宏观角度来看，在`ADK`框架的中一个会话的完整生命周期。如下图所示：（图片来源：https://google.github.io/adk-docs/runtime/#what-is-runtime）

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505261352643.png" width=80%></div>

&emsp;&emsp;如上图所示，`ADK` 框架会将每一次与用户的交互过程抽象为一个会话 `Session`，并通过*事件（Event）序列*来驱动 `Agent` 生命周期。我们先不深入技术细节，从宏观的角度来理解一下`ADK`各个组件都承担构建`Agent`应用时的哪些职责。

- **Agent 组件**

&emsp;&emsp;`Agent`组件的设计并不像`OpenAI Agents SDK`框架中的那样“静态”，即仅仅通过`Agent`组件构造的是一个代理对象（包括对该代理的模型选择、指令设置、外部工具或者`MCP`工具的调用等），具备自然语言理解、推理规划、内容生成和调用工具等能力。`ADK`会在`Agent`组件的设计中，也内置包含**运行时状态**，即接收用户的输入，并在内部处理生成最终的输入，或者调用外部工具的解析参数。同时也支持多种`Agent`对象类型，像`LLM Agent` 可以利用大模型根据指令和上下文动态决策下一步（包括是否使用工具或将任务转交其它 Agent），**Workflow Agent**可以按预定义流程执行，其子类包括顺序执行的 `SequentialAgent`、并行执行的 `ParallelAgent`、循环执行的 `LoopAgent` 等。

> &emsp;关于`Agent`的介绍与使用，我们将在后面的小节中详细展说明。

- **Session 组件**

&emsp;&emsp;`Session` 表示用户与 `Agent` 的一次完整对话，用来存储用户与 `Agent` 的对话历史、当前对话的上下文、对话的配置信息、对话的运行时状态等。`ADK`实现了不同形式的`Session`，比如临时的内存存储以及持久化的数据库存储以满足不同的应用场景。

- **Runner 组件与 Event 事件**

&emsp;&emsp;在 `ADK` 中，`Runner`是一个调度器，管理该会话中从用户输入到`Agent`输出的整个过程。核心职责包括：接收用户输入并将其引入 `Agent`、驱动 `Agent` 执行直到任务完成、调度处理执行过程中产生的事件，以及维护 `Agent` 执行的上下文与生命周期。而在调度过程中，是通过`Event`事件来驱动。**Event**是 `ADK` 中信息流通的基本单元，封装在对话过程中发生的各种内容：例如用户的提问消息、Agent 给出的回复、Agent 发出的工具使用请求（函数调用）、工具返回的结果、状态变化通知、控制信号以及错误信息等，因此大家可以简单理解为：`ADK` 框架是通过`Runner` 与 `Event` 的协作实现一个事件驱动的执行循环。

&emsp;&emsp;因此大家也能够基本感受到，`Runner` 和 `Event` 其实才是`ADK`区别于其他`Agent`开发框架的核心。用好`ADK`, 第一步就是要理解`Runner` 和 `Event` 的协作机制，而至于用什么大模型，构建多么复杂的`Agent`对象，全部都是基于`Runner` 和 `Event` 的协作机制来实现的。

&emsp;&emsp;因此，本节课程我们就先从`Runner`组件入手，详细介绍`ADK`框架的运行时机制，帮助大家清晰理解`ADK`框架的`Agent`设计模式。

# 二、ADK 代理运行时设计模式

&emsp;&emsp;正如上一小节所提到的，在 `ADK` 框架中，`Agent` 组件会负责决定是否调用工具以及生成调用参数，而 `Runner` 负责实际执行工具调用并将结果反馈给 `Agent`。通过这种设计模式去实现智能体的“思考”与“执行”的解耦。

&emsp;&emsp;`Runner`作为`ADK` 框架的运行管理模块，是让`Agent`能够“跑起来”的一个引擎。一个 `Runner Processor` 实例，（向左）接收用户输入并将其传递到`Agent`代理，（向右）与一个会话（`Session`）关联，管理该会话中从用户输入到`Agent`输出的整个过程。（向下）基于事件循环 (Event Loop) 运行，驱动 `Agent` 执行直到任务完成、调度处理执行过程中产生的事件，以及维护 `Agent` 执行的上下文与生命周期，这包括执行的工具调用、回调等之间的往返通信。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505261352643.png" width=80%></div>

&emsp;&emsp;一个 `Runner` 实例通常与一个会话（Session）关联，其执行流程是：在接收到用户的新请求时，`Runner` 首先会将该用户消息封装为一个 `Event`，通过 `SessionService` 保存到会话历史中，然后调用主 `Agent` 的入口方法启动事件循环，在 `Agent` 执行过程中，`Runner` 作为协作调度者持续等待 `Agent yield` 事件；每当获得一个事件，`Runner` 立即接管处理：利用各种服务组件对事件中记录的动作进行处理，如通过 `SessionService` 合并状态更新，直到当前会话状态这些内部处理完成后，`Runner` 将该事件标记为已处理并将其向上传递给应用的调用方（例如将回复发送给用户界面渲染），随后，`Runner` 通知 `Agent` 可以继续执行，从而进入下一个事件产生-处理循环。如此往复，直到 `Agent` 声明任务完成不再产生新事件，`Runner` 也就结束整个执行流程。


&emsp;&emsp;可以看出，`Runner` 模块扮演了的是orchestrator（协调器）的角色：一方面驱动和管理 `Agent` 的执行，另一方面负责所有外围事务（会话管理、工具执行、状态维护、结果输出等），实现控制流与业务逻辑的分离。

&emsp;&emsp;`Runner`是`ADK`框架运行的中枢，掌握`Runner`的运行机制其实就是应用的`ADK`框架的核心所在。同时也能从上图中看到，`Runner`并不是一个独立存在的组件，因此在学习的过程中，我们会结合`Runner`组件自身的构造原理以及与其他组件的关联，来综合理解`ADK`框架的设计模式。

&emsp;&emsp;首先，我们先来关注`Runner`组件的构造方法，其源码位置：https://github.com/google/adk-python/blob/main/src/google/adk/runners.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505281017614.png" width=60%></div>


&emsp;&emsp;基于源码可以看到在`Runner`的构造函数中，可以接收如下参数，我们整理成表格如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Runner 构造函数参数</font></p>
<div class="center">

| 参数名               | 类型                          | 描述                             | 必填/可选 |
|---------------------|-------------------------------|----------------------------------|-----------|
| `app_name`          | `str`                         | 运行器的应用名称                 | <font color="red">必填</font>      |
| `agent`             | `BaseAgent`                  | 要运行的初始代理                   | <font color="red">必填</font>      |
| `artifact_service`  | `Optional[BaseArtifactService]` | 运行器的文件管理（可选）         | 可选      |
| `session_service`   | `BaseSessionService`         | 运行器的会话服务                 | <font color="red">必填</font>      |
| `memory_service`    | `Optional[BaseMemoryService]` | 运行器的内存服务（可选）         | 可选      |

&emsp;&emsp;因此在构造`Runner`实例对象时，其基本的实现形式如下所示。这里我们使用`liteLLM`接入`DeepSeek`模型，注意需要在同级目录下创建`.env`文件，并配置`DS_API_KEY`和`DS_BASE_URL`。同时，对于`Agent`的构建，我们使用最简模式，即只设置`name`和`model`。（我们在下一小节课程再详细介绍`Agent`组件的高阶应用）

In [5]:
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
import os
from dotenv import load_dotenv
load_dotenv(override=True)

DS_BASE_URL = os.getenv("DEEPSEEK_API_BASE")
DS_API_KEY = os.getenv("DEEPSEEK_API_KEY")

# 1. 通过 litellm 接入 DeepSeek 模型
model = LiteLlm(
    model="deepseek/deepseek-chat",  
    api_base=DS_BASE_URL,
    api_key=DS_API_KEY
)

# 2. 使用最简模式构建一个Agent代理
init_agent = Agent(
    name="chatbot",
    model=model,
    instruction="你是一个乐于助人的中文助手。",
)

&emsp;&emsp;同样，因为`session_service`作为`Runner`的必填参数，我们这里先使用`InMemorySessionService`作为示例接入。在接下来的课程中再详细介绍更加灵活的`SessionService`构建方法。

In [6]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

APP_NAME = "chatbot"  # 手动定义应用名称
USER_ID = "user_1"    # 手动定义用户ID
SESSION_ID = "session_001"  # 手动定义会话ID


# 3. 创建内存会话缓存
session_service = InMemorySessionService()

# 4. 在内存中实际创建缓存数据
await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)


# 5. 创建Runner实例对象
runner = Runner(
    app_name="chatbot",
    agent=init_agent,
    session_service=session_service
)

&emsp;&emsp;在`Runner`实例对象创建后，如果涉及运行启动，则需要调用其内部的方法，目前`Runner`类中提供的运行方法如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505281201487.png" width=60%></div>

&emsp;&emsp;其中`run_async()` 是实际运行`Runner`的主要入口点,`run()` 仅为同步方法，在内部调用异步版本。实际应用中，我们通常使用`run_async()`方法来启动`Runner`的运行。在运行过程中，其工作流程如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505301316424.png" width=60%></div>

&emsp;&emsp;如上图所示，`ADK（Agent Development Kit）`采用高度模块化的事件循环机制来管理整个代理的执行流。其核心流程分为四个阶段：<font color='red'>初始化阶段、代理执行阶段、事件处理阶段和恢复执行阶段</font>，涵盖服务组件的数据流转。

&emsp;&emsp;首先，在初始化阶段，通过 `Runner.run_async()` 接收用户传递的请求，由 `SessionService` 加载或新建会话（`Session`），接着为本次请求创建执行上下文（`InvocationContext`），并把用户的输入消息添加到会话历史中，用于后续的对话过程，执行完以上流程后，进入正式的代理业务处理逻辑。

&emsp;&emsp;进入代理执行阶段后，会调用`Agent`代理的 `run_async()` 方法，正式进入业务逻辑处理。此过程中，代理会根据具体的业务推理和工具调用，动态地生成一个或多个 `Event（事件）`对象。每个 `Event` 代表代理推理的一个关键节点，事件中包含当前状态及响应内容，同时代理执行会在此阶段暂停，等待事件处理流程。

&emsp;&emsp;在事件处理阶段，`Runner` 会捕获代理产出的 `Event`。随后，系统会通过 `SessionService.commit_state_delta()` 将本轮的状态变更持久化，确保会话和流程状态得以保存；通过 `ArtifactService.save_artifact()` 将本次事件涉及的文件、媒资或其他工件（artifact）同步保存。同时，事件会被追加到会话历史当中，便于查询追溯，并可以向上游（如前端 UI）实时转发，实现可视化反馈和交互。

&emsp;&emsp;事件处理完成后，进入恢复执行阶段，代理会从暂停点恢复执行，继续后续推理和处理。此时，代理能够实时读取到所有已提交的状态变更，并以最新上下文为基础继续决策。系统会判断是否已经获得最终响应（通过 is_final_response() 检查），如果流程还未结束，则继续事件循环，直到所有流程节点处理完成。如果遇到特殊情况，例如需跳过总结或涉及长时间运行的工具调用，则会进入相应的分支逻辑进行处理。

&emsp;&emsp;从整体上理解，`Runner` 持续上述循环，直到 `Agent` 执行完毕不再产生事件。这种结构遵循典型的生产-消费循环模式，其中 `Agent` 是事件的生产者，`Runner` 是事件的消费者和处理者。同时，`Runner` 也充当任务调度器，通过协调 `Agent` 执行顺序、调用服务处理副作用、以及控制何时将结果输出或何时继续 `Agent` 执行，并且整个机制依赖于 `InvocationContext`、`SessionService` 和 `ArtifactService` 等服务组件的协同。`InvocationContext` 负责具体的执行上下文和参数配置，`SessionService` 负责会话级的状态管理和历史维护，`ArtifactService` 则负责文件、媒资及其他工件的全生命周期管理。通过事件驱动、状态持久化和工件管理三大核心服务组件实现高可扩展和高可靠的多轮对话及复杂业务流程。


&emsp;&emsp;理解了上述`ADK`的运行机制后，我们首先看`Runner`中`run_async()`方法的构造函数参数，如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>run_async 构造函数参数</font></p>
<div class="center">

| 参数名           | 类型                          | 描述                             |
|------------------|-------------------------------|----------------------------------|
| `user_id`        | `str`                         | 会话的用户 ID                    |
| `session_id`     | `str`                         | 会话的 ID                        |
| `new_message`    | `types.Content`              | 要添加到会话的新消息             |
| `run_config`     | `RunConfig`                  | 代理的运行配置（默认为 `RunConfig()`） |

&emsp;&emsp;其中`user_id`和`session_id`用于`SessionService`中会话的创建和维护。`new_message`是用户输入的消息内容，作用于`InvocationContext`中提供`Agent`执行过程中的上下文信息。而`run_config`是代理的运行配置，作为非必填参数用于更加灵活的控制`Runner`的运行配置。这里的一个关键点是`new_message`，其类型为`types.Content`，其源码定义如下：

```python
    class Content(_common.BaseModel):
    """Contains the multi-part content of a message."""

        parts: Optional[list[Part]] = Field(
            default=None,
            description="""List of parts that constitute a single message. Each part may have
            a different IANA MIME type.""",
        )
        role: Optional[str] = Field(
            default=None,
            description="""Optional. The producer of the content. Must be either 'user' or
            'model'. Useful to set for multi-turn conversations, otherwise can be
            empty. If role is not specified, SDK will determine the role.""",
        )
```

&emsp;&emsp;这里的`Types`是 `Google ADK`框架中的类型定义，用于表示消息序列的类型。正常对于`OpenAI Chat API` 标准格式下消息对话形式是这样的：

```python
    {
        "role": "sytem",
        "content": "你是一个乐于助人的中文助手。"

        "role": "user",
        "content": "你好，请你介绍一下你自己。"

        "role": "assistant",
        "content": "你好，我是一个乐于助人的中文助手。"

        # .. 如果涉及工具调用，则需要添加工具调用的内容
        "role": "tool",
        "content":{
            "type": "tool_call",
            "tool_call_id": "tool_call_id",
            "parts": [
                {
                    "type": "text",
                    "text": "工具调用结果"
                }
            ]
        }
    }
```

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>GPT Vs Gemini</font></p>
<div class="center">

| 特性     | OpenAI GPT 模型                               | Google Gemini 模型                           |
| ------ | ------------------------------------------- | ------------------------------------------ | 
| 消息格式   | 结构化消息序列（`system`、`user`、`assistant`、`tool`） | 数组形式的消息（`contents` 数组，包含 `role` 和 `parts`） |                                                     
| 输出格式控制 | 通过 `system` 消息设定行为                          | 通过 `responseSchema` 配置输出格式                 |                                                     
| 多模态支持  | 支持（如 GPT-4o）                                | 原生支持（文本、图像、音频等）                            |                                                     
| 集成方式   | 通过 OpenAI API 或 Azure OpenAI 服务             | 通过 Vertex AI 或 Google Gen AI SDK           |  |



&emsp;&emsp;大部分模型，无论是开源还是闭源，主流遵循的消息格式还是上述`OpenAI Chat API`的规范。所以对`ADK`框架来说，`Types.Content`类的抽象的作用就是将常规的输入文本适配到`ADK`框架中。因此在执行`Runner.run_async()`方法时，则需要按照`ADK`框架的规范来构造`new_message`参数。如下代码所示：

In [7]:
from google.genai import types

query = "你好，请你介绍一下你自己。"

# 将用户的问题转换为 ADK 框架的兼容格式
content = types.Content(
    role='user', 
    parts=[types.Part(text=query)]
    )

# 异步运行
async for event in runner.run_async(
    user_id="user_1",
    session_id="session_001", 
    new_message=content,
):
    # 处理事件
    print(event)


content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='你好！我是一个人工智能助手，旨在为你提供各种帮助和服务。我可以回答问题、提供建议、协助学习、处理文本、分析数据，甚至陪你聊天解闷。  \n\n我的特点包括：  \n1. **多领域知识**：覆盖科技、历史、文化、生活等多个领域。  \n2. **语言处理**：支持中文和多种语言交流，能帮你翻译或润色文本。  \n3. **任务协助**：比如日程提醒、简单计算、信息查询等。  \n4. **持续学习**：虽然我的知识截至训练数据的时间点，但会尽量提供最新信息。  \n\n我没有情感或主观意识，但会以友好、专业的方式与你互动。如果有任何需求或问题，随时告诉我！ 😊  \n\n你想让我帮你做点什么呢？')], role='model') grounding_metadata=None partial=False turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=161, candidates_tokens_details=None, prompt_token_count=30, prompt_tokens_details=None, thoughts_token_count=None, tool_use_prompt_token_count=None, tool_use_prompt_tokens_details=None, total_token_count=191, traffic_t

&emsp;&emsp;对单次运行返回的数据对象，`Runner.run_async()`方法会返回一个`Event`对象的迭代器，因此我们想要清晰的知道返回的数据结构，则需要对`Event`对象的结构进行了解。因为`Runner`是通过**事件循环**与 `Agent`做的交互，而每一次关键操作都会被结构化为事件（`Event`）。

# 三、Event 事件类型及应用

&emsp;&emsp;事件（Event）是`ADK`中信息流的基本单元，承载代理交互生命周期中发生的每一次重要事件。在 `ADK`框架的设计模式中，事件驱动机制贯穿 `Agent` 执行的始终。`Agent` 的一切输出（无论是对用户的自然语言回答还是工具调用请求）都会被包装成 `Event` 并依次 `yield` 给 `Runner` 处理。`Runner` 对事件进行调度处理：例如，当 `Event` 指示需要调用某个工具时，`Runner` 会读取其中的 `action` 并实际调用相应工具函数，然后将工具执行结果再次封装为一个新的 `Event`供 `Agent` 后续处理。可以认为，每个 `Event` 就是一次 `Agent` 与外部环境（包括用户、工具和其他 Agent）的交互契机：它既携带了交互的内容，也携带了对框架的“请求”或“信号”。其完整的`Event`事件流如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505301318663.png" width=60%></div>


&emsp;&emsp;事件流在系统中按顺序传递，例如，一个工具调用事件必须先由 `Runner` 完成工具执行并生成结果事件，`Agent` 才能继续推进自己的逻辑。通过这种事件调度机制，`ADK` 将智能体的连续对话过程离散化为一系列事件流，使得对话历史、状态演变和操作副作用都被显式地记录和管理。

&emsp;&emsp;从初始用户输入到最终响应，以及其间的所有步骤信息，包括：

- **内容载体**：承载用户消息、模型回复、工具调用等，一切数据类型都以 `Event` 类型流动；
- **状态变更**：通过`actions`(事件行为)传递状态修改指令，比如状态发生了变更、代理发生了转移、执行持久化存储数据等；
- **执行控制**：控制代理的执行流程和转移，确定接下来运行哪个代理或循环是否应该终止；
- **元数据管理**：记录事件的时间戳、来源、类型等，提供完整交互、按时间顺序排列的历史记录，用于逐步调试、审计和了解代理行为；

&emsp;&emsp;可以简单理解为：从用户的查询到代理的最终答案的整个过程都是通过 `Event` 对象的生成、解释和处理来协调的。其源码位置：https://github.com/google/adk-python/blob/main/src/google/adk/events/event.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505281257059.png" width=60%></div>

&emsp;&emsp;从源码中可以看到，`Event(LlmResponse) -> BaseModel(Pydantic)`, 即`Event`继承自`LlmResponse`，所以`Event`不仅是一个事件容器，还包含了`LlmResponse`中定义的所有大模型的响应信息。因此其核心字段可以分为基础标识、内容标识和行为控制标识，如下表所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>基础标识字段</font></p>
<div class="center">

| 参数名           | 类型                          | 描述                             |
|------------------|-------------------------------|----------------------------------|
| `invocation_id`        | `str`                         | 调用 ID，关联到特定的执行上下文                    |
| `author`     | `str`                         | 事件作者："user" 或代理名称                        |
| `id`    | `str`              | 事件唯一标识符（自动生成）             |
| `timestamp`     | `float`                  | 事件时间戳(自动生成) |

&emsp;&emsp;`author`字段可以是`user`, 表示直接来自用户的输入。也可以是`AgentName`, 表示特定代理的输出或者操作。这里在`ADK`框架中匹配的就是<font color="red">**用户消息事件**</font>，如下代码所示：

In [8]:
from google.adk.events import Event

# 用户输入
user_event = Event(
    author="user",
    content=types.Content(
        role="user",
        parts=[types.Part(text="你好，请帮我查询天气")]
    )
)

In [9]:
print(f"用户消息事件的作者为：{user_event.author}")
print(f"用户消息事件id为：{user_event.id}")
print(f"用户消息事件时间戳为：{user_event.timestamp}") 

用户消息事件的作者为：user
用户消息事件id为：ESEBiDdn
用户消息事件时间戳为：1748582510.378752


<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>内容标识字段（继承自 LlmResponse）</font></p>
<div class="center">

| 参数名           | 类型                          | 描述                             |
|------------------|-------------------------------|----------------------------------|
| `content`        | `Optional[types.Content]`                         | 消息内容                    |
| `grounding_metadata`     | `Optional[types.GroundingMetadata]`                         | 基础元数据                        |
| `partial`    | `Optional[bool]`              | 是否为流式响应的部分内容             |
| `turn_complete`     | `Optional[bool]`                  | 回合是否完成 |
| `error_code`        | `Optional[str]`                         | 错误代码                    |
| `error_message`     | `Optional[str]`                         | 错误消息                        |
| `usage_metadata`    | `Optional[types.GenerateContentResponseUsageMetadata]`              | 使用统计             |


&emsp;&emsp;其中`content`字段是`Event`事件中承载消息内容的核心字段，其类型为`types.Content`，其源码定义如下：

```python
    class Content(_common.BaseModel):
    """Contains the multi-part content of a message."""

        parts: Optional[list[Part]] = Field(
            default=None,
            description="""List of parts that constitute a single message. Each part may have
            a different IANA MIME type.""",
        )
        role: Optional[str] = Field(
            default=None,
            description="""Optional. The producer of the content. Must be either 'user' or
            'model'. Useful to set for multi-turn conversations, otherwise can be
            empty. If role is not specified, SDK will determine the role.""",
        )
```

&emsp;&emsp;在`Content`类型中，`parts`字段是`Part`类型的列表，`Part`类型定义如下：

```python
    class Part(_common.BaseModel):
    """A datatype containing media content.

    Exactly one field within a Part should be set, representing the specific type
    of content being conveyed. Using multiple fields within the same `Part`
    instance is considered invalid.
    """

        video_metadata: Optional[VideoMetadata] = Field(
            default=None, description="""Metadata for a given video."""
        )
        thought: Optional[bool] = Field(
            default=None,
            description="""Indicates if the part is thought from the model.""",
        )
        inline_data: Optional[Blob] = Field(
            default=None, description="""Optional. Inlined bytes data."""
        )
        code_execution_result: Optional[CodeExecutionResult] = Field(
            default=None,
            description="""Optional. Result of executing the [ExecutableCode].""",
        )
        executable_code: Optional[ExecutableCode] = Field(
            default=None,
            description="""Optional. Code generated by the model that is meant to be executed.""",
        )
        file_data: Optional[FileData] = Field(
            default=None, description="""Optional. URI based data."""
        )
        function_call: Optional[FunctionCall] = Field(
            default=None,
            description="""Optional. A predicted [FunctionCall] returned from the model that contains a string representing the [FunctionDeclaration.name] with the parameters and their values.""",
        )
        function_response: Optional[FunctionResponse] = Field(
            default=None,
            description="""Optional. The result output of a [FunctionCall] that contains a string representing the [FunctionDeclaration.name] and a structured JSON object containing any output from the function call. It is used as context to the model.""",
        )
        text: Optional[str] = Field(
            default=None, description="""Optional. Text part (can be code)."""
        )
```

&emsp;&emsp;除了`Google`内置的一些应用外，在`Part`类型中，通用的内容主要关注的如下三个方面：

1. **text**： 表示对话消息；
2. **function_call**：表示是否存在工具调用请求，如果存在，内部会包含`.name`和`.args`参数；
3. **function_response**：表示工具调用结果，如果存在，内部会包含`.name`和`.resonse`参数；

&emsp;&emsp;因此，对于内容标识字段，在`ADK`框架中匹配的就是<font color=red>**代理响应事件**</font>，如下代码所示：

In [10]:
# 代理文本响应
agent_event = Event(
    author="weather_agent",
    content=types.Content(
        role="model",
        parts=[types.Part(text="我来帮您查询天气信息")]
    )
)

In [11]:
user_event

Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='你好，请帮我查询天气')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='ESEBiDdn', timestamp=1748582510.378752)

In [12]:
print(f"代理响应事件的作者为：{agent_event.author}")
print(f"代理响应事件内容为：{agent_event.content.parts[0].text}")
print(f"代理是否存在执行工具：{agent_event.content.parts[0].function_call}")

代理响应事件的作者为：weather_agent
代理响应事件内容为：我来帮您查询天气信息
代理是否存在执行工具：None


&emsp;&emsp;如果代理响应涉及工具调用，则`function_call`字段会存在，其对应的是<font color=red>**工具调用事件**</font>，如下代码所示：

In [13]:
# 工具调用
tool_call_event = Event(
    author="weather_agent",
    content=types.Content(
        role="model",
        parts=[types.Part(
            function_call=types.FunctionCall(
                name="get_weather",
                args={"city": "北京"}
            )
        )]
    )
)

In [14]:
print(f"代理响应事件的作者为：{tool_call_event.author}")
print(f"代理响应事件内容为：{tool_call_event.content.parts[0].text}")
print(f"代理是否存在执行工具：{tool_call_event.content.parts[0].function_call}")
print(f"代理执行工具的结果为：{tool_call_event.content.parts[0].function_response}")

代理响应事件的作者为：weather_agent
代理响应事件内容为：None
代理是否存在执行工具：id=None args={'city': '北京'} name='get_weather'
代理执行工具的结果为：None


&emsp;&emsp;大模型函数调用的标准流程，是由大模型根据用户的输入意图，在工具列表中选择匹配的工具并根据该工具的`Json Schema`表示生成对应的参数，实际的去执行该工具，并将执行结果返回给大模型后，大模型才能继续根据执行结果进行后续的推理。因此，这里匹配的是<font color=red>**工具响应事件**</font>。如下代码所示：

In [15]:
from google.genai.types import FunctionResponse

tool_response_event = Event(
        author="weather_agent", 
        content=types.Content(
            role="tool",  # 工具响应角色是 tool
            parts=[types.Part(function_response=types.FunctionResponse(name="get_weather", response={"temperature": "25°C", "condition": "晴天"}))]
        ),
    )

In [16]:
print(f"代理响应事件的作者为：{tool_response_event.author}")
print(f"代理响应事件内容为：{tool_response_event.content.parts[0].text}")
print(f"代理是否存在执行工具：{tool_response_event.content.parts[0].function_call}")
print(f"代理执行工具的结果为：{tool_response_event.content.parts[0].function_response}")

代理响应事件的作者为：weather_agent
代理响应事件内容为：None
代理是否存在执行工具：None
代理执行工具的结果为：will_continue=None scheduling=None id=None name='get_weather' response={'temperature': '25°C', 'condition': '晴天'}


<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>行为控制标识字段</font></p>
<div class="center">

| 参数名           | 类型                          | 描述                             |
|------------------|-------------------------------|----------------------------------|
| `actions`        | `EventActions`                         | 事件携带的动作指令                    |
| `long_running_tool_ids`     | `Optional[set[str]]`                         | 长时间运行的工具 ID                        |
| `branch`    | `Optional[str]`              | 代理分支路径             |      |


&emsp;&emsp;`EventActions` 是`Event`事件中携带的动作指令，是`Event`的核心组成，定义了事件可以触发的各种动作：

```python
    class EventActions(BaseModel):
        # 状态管理
        state_delta: dict[str, object] = {}           # 状态变更增量
        artifact_delta: dict[str, int] = {}          # 工件变更增量
        
        # 流程控制
        transfer_to_agent: Optional[str] = None       # 转移到指定代理
        escalate: Optional[bool] = None               # 升级到上级代理
        
        # 处理控制
        skip_summarization: Optional[bool] = None     # 跳过摘要生成
        
        # 认证管理
        requested_auth_configs: dict[str, AuthConfig] = {}  # 请求的认证配置
```

&emsp;&emsp;比如对于函数调用事件来说，就是需要通过`EventActions`中的`state_delta`字段来更新执行外部工具后返回的实时数据，因此代码如下所示：

In [17]:
from google.adk.events import EventActions

state_update_event = Event(
        author="weather_agent", 
        content=types.Content(
            role="tool  ",  # 工具响应角色是 tool
            parts=[types.Part(function_response=types.FunctionResponse(name="get_weather", response={"temperature": "25°C", "condition": "晴天"}))]
        ),
        actions=EventActions(
        state_delta={
            "last_query_city": "北京",
            "last_query_time": "2025-05-28 15:00:00"
        }
    )
    )

In [18]:
print(f"代理响应事件的作者为：{state_update_event.author}")
print(f"代理响应事件内容为：{ state_update_event.content.parts[0].text}")
print(f"代理是否存在执行工具：{state_update_event.content.parts[0].function_call}")
print(f"代理执行工具的结果为：{state_update_event.content.parts[0].function_response}")
print(f"代理执行工具的状态更新为：{state_update_event.actions.state_delta}")

代理响应事件的作者为：weather_agent
代理响应事件内容为：None
代理是否存在执行工具：None
代理执行工具的结果为：will_continue=None scheduling=None id=None name='get_weather' response={'temperature': '25°C', 'condition': '晴天'}
代理执行工具的状态更新为：{'last_query_city': '北京', 'last_query_time': '2025-05-28 15:00:00'}


&emsp;&emsp;以上就是在`ADK`框架中，一个包含工具调用的`Agent`的完整执行。所有组件间通信都通过 `Event` 进行， 通过 `actions.state_delta` 实现状态的原子性更新（要么状态更新成功并被持久化，要么更新失败，状态保持不变）， 以及通过 `is_final_response()` 明确回合边界。

```python
    用户输入 → Event(author="user") 
        ↓
    代理处理 → Event(author="agent", function_call)
        ↓
    工具执行 → Event(author="agent", function_response)
        ↓
    状态更新 → Event(author="agent", state_delta)
        ↓
    最终响应 → Event(author="agent", is_final=True)
```

&emsp;&emsp;以上是我们结合`ADK`的设计理念构造的分布执行流程，相信大家已经对`ADK`中的不同事件有了相对清晰的理解。现在再来看`run_async`方法执行`Agent`时，其实就是通过这个方法自动生成上述`Event`事件并进行处理。其执行流程如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505301342444.png" width=60%></div>

&emsp;&emsp;所以当我们了解了`ADK`中的事件类型，自然就可以非常清晰且准确的提取和访问`Runner`过程中不同`Event`产生的相关数据。

In [19]:
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm

import os
from dotenv import load_dotenv
load_dotenv(override=True)


DS_API_KEY = os.getenv("DS_API_KEY")
DS_BASE_URL = os.getenv("DS_BASE_URL")

model = LiteLlm(
    model="deepseek/deepseek-chat",  
    api_base=DS_BASE_URL,
    api_key=DS_API_KEY
)

init_agent = Agent(
    name="chabot",
    model=model,
    instruction="你是一位乐于助人的智能助手，请根据用户的问题给出最合适的回答。"
)

In [20]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

APP_NAME = "chatbot"
USER_ID = "user_1"
SESSION_ID = "session_001"

session_service = InMemorySessionService()
await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)

Session(id='session_001', app_name='chatbot', user_id='user_1', state={}, events=[], last_update_time=1748588825.1687121)

In [21]:
runner = Runner(
    app_name="chatbot",
    agent=init_agent,
    session_service=session_service
)

In [22]:
from google.genai import types


query = "你好，请你介绍一下你自己。"

# 将用户的问题转换为 ADK 格式
content = types.Content(role='user', parts=[types.Part(text=query)])

# 异步运行
async for event in runner.run_async(
    user_id="user_1",
    session_id="session_001", 
    new_message=content,
):
    
    print(f"当前事件来自: {event.author}")
    # 访问文本内容
    if event.content and event.content.parts:
        print(f"当前事件内容为: {event.content.parts[0].text}")

当前事件来自: chabot
当前事件内容为: 你好！我是你的智能助手，内部代号为"chabot"。我可以帮助你解答各种问题、提供信息、协助完成任务或只是陪你聊聊天。无论是学习、工作还是生活中的疑问，我都会尽力为你提供准确和有用的回答。我没有实体，但随时在线为你服务。你可以随时向我提问或提出需求，我会以专业、友好的态度回应你。请问有什么我可以帮你的吗？


- 添加外部工具

In [23]:
import requests

def get_weather(city: str) -> str:
    """
    Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "Beijing", "Shanghai").
                   Note: For cities in China, use the city's English name (e.g., "Beijing").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": city,               
        "appid": os.getenv("OPENWEATHER_API_KEY"),    # 输入自己的API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    city = data.get("name", "未知")
    country = data.get("sys", {}).get("country", "未知")
    temp = data.get("main", {}).get("temp", "N/A")
    humidity = data.get("main", {}).get("humidity", "N/A")
    wind_speed = data.get("wind", {}).get("speed", "N/A")
    weather_list = data.get("weather", [{}])
    description = weather_list[0].get("description", "未知")

    return (
        f"🌍 {city}, {country}\n"
        f"🌡 温度: {temp}°C\n"
        f"💧 湿度: {humidity}%\n"
        f"🌬 风速: {wind_speed} m/s\n"
        f"🌤 天气: {description}\n"
    )

In [24]:
get_weather(city="Beijing")

'🌍 Beijing, CN\n🌡 温度: 29.94°C\n💧 湿度: 31%\n🌬 风速: 5.98 m/s\n🌤 天气: 阴，多云\n'

In [25]:
weather_agent = Agent(
    name="weather_agent",
    model=model, 
    description="用于进行某地天气查询的Agent智能体",
    instruction="你是一个有帮助的天气助手。"
                "当用户询问特定城市的天气时，"
                "使用 'get_weather' 工具查找相关信息。"
                "如果工具返回错误，礼貌地告知用户。",
    tools=[get_weather], 
)

In [26]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

APP_NAME = "chatbot"
USER_ID = "user_1"
SESSION_ID = "session_001"

session_service = InMemorySessionService()
await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)

Session(id='session_001', app_name='chatbot', user_id='user_1', state={}, events=[], last_update_time=1748588887.893337)

In [27]:
runner = Runner(
    agent=weather_agent, 
    app_name=APP_NAME,   
    session_service=session_service 
)

In [28]:
query = "你好，帮我查询一下上海现在的天气情况。"

# 将用户的问题转换为 ADK 格式
content = types.Content(role='user', parts=[types.Part(text=query)])


# 异步运行
async for event in runner.run_async(
    user_id="user_1",
    session_id="session_001", 
    new_message=content,
):
    
    print(f"当前事件来自: {event.author}")
    # 访问文本内容
    if event.content and event.content.parts:    
        # 如果事件中存在工具调用，则打印执行的工具和参数
        if event.get_function_calls():
            print(f"需要执行外部工具:")
            for call in event.get_function_calls():
                tool_name = call.name
                arguments = call.args 
                print(f"  执行的工具是: {tool_name}, 传递的参数是: {arguments}")


        # 如果存在工具调用且工具调用成功，则打印工具调用的结果
        if event.get_function_responses():
            print(f"工具调用成功:")
            for response in event.get_function_responses():
                print(f"  工具：{response.name} 调用的结果是: {response.response}")

        elif event.content.parts[0].text:
            print(f"最终响应: {event.content.parts[0].text}")

当前事件来自: weather_agent
需要执行外部工具:
  执行的工具是: get_weather, 传递的参数是: {'city': 'Shanghai'}
当前事件来自: weather_agent
工具调用成功:
  工具：get_weather 调用的结果是: {'result': '🌍 Shanghai, CN\n🌡 温度: 29.92°C\n💧 湿度: 28%\n🌬 风速: 4 m/s\n🌤 天气: 晴\n'}
当前事件来自: weather_agent
最终响应: 上海当前的天气情况如下：

- **温度**: 29.92°C
- **湿度**: 28%
- **风速**: 4 m/s
- **天气**: 晴

希望这些信息对您有帮助！


&emsp;&emsp;接下来我们进一步地看一下如何做流式输出

# 四、流式输出及FastAPI接口封装

&emsp;&emsp;`Google ADK` 中实现流式输出与如`OpenAI Agents SDK`框架等不同，`ADK` 提供对流式响应的内置支持，我们只需要监测`Event`事件中的`partial`字段，即可实时的提取到增量产生的`Token`。当然，默认对于`run_async`方法是不开启流式输出功能的，我们需要在配置`Runner`时，通过`RunConfig`指定流式输出的模式。目前`ADK`支持指定两种流式化输出模式，分别是：`SSE` 和 `BIDI (Bidirectional)`，源码位置如下：https://github.com/google/adk-python/blob/main/src/google/adk/agents/run_config.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505301537469.png" width=60%></div>

&emsp;&emsp;其中`SSE` 模式是使用标准的`Server-Sent Events`协议，单向的服务器向客户端发送流式数据，主要用于文本内容的逐步发送。而`BIDI (Bidirectional) `模式是使用`WebSocket`协议实现双向通信,双向的客户端和服务器可以同时发送数据。但是目前通过`LiteLlm`接入的模型还不支持`BIDI`模式。只有使用`Google`的`Gemini`模型（google_llm.py）才能够进行使用。其配置方法如下所示：

```python
    # 配置流式模式
    run_config = RunConfig(streaming_mode=StreamingMode.SSE)

    # 执行代理运行的流式输出
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=content,
        run_config=run_config
    ):  
```

&emsp;&emsp;这里大家可以通过`stream_event.py`文件查看完整的实现逻辑。其运行效果如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505301607729.png" width=60%></div>

&emsp;&emsp;同时，也基于`ADK`的流式接口封装了`FastAPI`接口，大家可以直接进行启动并通过`Http Request`进行服务访问。具体实现代码文件为`stream_api.py`，启动方法如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505301621655.png" width=60%></div>

&emsp;&emsp;启动后，即可在浏览器页面中输入`localhost:8000`进行接口文档的查看或者测试。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505301621654.png" width=60%></div>